## Estimating Maximum Likelihood

In [1]:
from ngram_counts import count_ngrams
from base_ngram_model import BaseNgramModel

In [2]:
import pandas as pd
from collections import Counter
def create_vocab_sents():
    df = pd.read_csv('../data/corpus.csv', header=None)
    df = df[df[0].str.len() > 20]
    
    text = df[0][:1000].values
    sents = [t.split() for t in text]
    tokens = [j for i in sents for j in i]
    vocab = Counter(tokens)
    return vocab, sents

vocab, sents = create_vocab_sents()

In [4]:
trigram_model = BaseNgramModel(count_ngrams(3, vocab, sents))
print(trigram_model.perplexity(sents[0]))

In [6]:
fivegram_model = BaseNgramModel(count_ngrams(5, vocab, sents))
print(fivegram_model.perplexity(sents[0]))

1.3934211547457294


## Smoothing
To deal with the problem that words in our vocabulary may appear in a test set in an unseen context (e.g. appear after a word they never appeared after in training). 

To prevent language model from assigning zero probability to these unseen events, we'll have to shave off a bit of probability mass from some more frequent events and give it to the events we've never seen. This process is called **smoothing** or **discounting**. 

### Laplace Smoothing
The simplest way to do smoothing is to add one to all the bigram counts, before we normalize them into probabilities. All the counts that used to be zero will now have a count of 1, the counts of 1 will be 2, and so on. 

Laplace smoothing does not perform well enough to be used in model n-gram models, but we use it to illustrate the concept.

## Add-k smoothing
One algernative to add-one smoothing is to move a bit less of the probability mass from the seen to the unseen events. Instead of adding 1 to each count, we add a fractional count *k* (0.5? 0.05? 0.01?). This algorithm is therefore called **add-k smoothing**.

$$
P^*_{\mathrm{Add-k}}(w_n|w_{n-1}) = \frac{C(w_{n-1}w_n) + k}{C(w_{n-1})+kV},
$$
where $V$ is the vocabulary and $C(*)$ is the count.

Add-k smoothing requires that we have a method for choosing *k$; this can be done, for example, by optimizing on a **devset**. Although add-k is useful for some tasks (including text classification), it turns out that it still doesn't work well for language modelling, generating counts with poor variances and often inappropriates discounts.

## Backoff and Interpolation
So far we are trying to resolve the problem of zero frequency n-grams. But there is an additional source of knowledge we can draw on. If we are trying to compute $P(w_n|w_{n-2}w_{n-1})$ but we have no examples of a particular trigram $w_{n-2}w_{n-1}w_n$. We can instead estimate its probability by using the bigram probability $P(w_n|w_{n-1})$. Similarly, if we don't have counts to compute $P(w_n|w_{n-1})$, we can look to the unigram $P(w_n)$.

In other words, sometimes using **less context** is a good thing, helping to generalize more for contexts that the model hasn't learned much about. There are two ways to use this n-gram "hierarchy". 

### Backoff
We use the trigram if the evidence is sufficient, otherwise we use the bigram, otherwise the unigram. I.e. we only 'back off' to a lower-order n-gram if we have zero evidence for a higher-order n-gram.

### Interpolation
We always mix the probability estimates from all the n-gram estimators, weighting and combining the trigram, bigram, and unigram counts.

Simple linear interpolation: we combine different order n-grams by linearly interpolating all the model. Thus, we estimate the trigram probability $P(w_n|w_{n-2}w_{n-1})$ by mixing together the unigram, bigram and trigram probabilities, each weighted by a $\lambda$:

$$
\begin{array}
\hat{P}(w_n|w_{n-2}w_{n-1})  & = \lambda_1 P(w_n|w_{n-2}w_{n-1})\\
 & + \lambda_2 P(w_n|w_{n-1})\\
 & + \lambda_3 P(w_n)
 \end{array}
 $$
 such that $\lambda$s sum to 1:
 
 $$
 \sum_i\lambda_i = 1
 $$
 
In a slightly more sophisticated version of linear interpolation, each $\lambda$ weight is computed by conditioning on the context. This way, if we have particularly accurate accounts for a particular bigram, we assume that the counts of the trigrams based on this bigram will be more trustworthy, so we can make the $\lambda$s for those trigrams higher and thus give that trigram more weight in the interpolation. 

$$
\begin{array}
\hat{P}(w_n|w_{n-2}w_{n-1}) & = \lambda_1(w_{n-2}^{n-1}) P(w_n|w_{n-2}w_{n-1})\\
 & + \lambda_2(w_{n-2}^{n-1})P(w_n|w_{n-1})\\
 & + \lambda_3(w_{n-2}^{n-1})P(w_n)
\end{array}
$$

How are these $\lambda$ values set? Both the simple interpolation and conditional interpolation $\lambda$s are learned from a **held-out** corpus. A held-out corpus is an additional training corpus that we use to set hyperparameters like these $\lambda$ values, by choosing the $\lambda$ values that maximize the likelihood of the held-out corpus. 

That is, we fix the n-gram probabilities and then search for the $\lambda$ values that when plugged into the linear equations give us the highest probability of the held-out set. There are various ways to find the optimal set of $\lambda$s. One way is to use the EM algorithm, an iterative learning algorithm that converges on locally optimzal $\lambda$s.

In a **backoff** n-gram model, if the n-gram we need has zero counts, we approximate it by backing off to the (N-1)-gram. We continue backing off until we reach a history that has some counts.

In order for a backoff model to give a correct probability distribution, we have to **discount** the higher-order n-grams to save some probability mass for the lower order n-grams. Just as with add-one smoothing, if the higher-order n-grams aren't discounted and we just used the undiscounted MLE probability, then as soon as we replaced an n-gram which has zero probability with a lower-order n-gram, we could be adding probability mass, and the total probability assigned to all possible strings by the language model would be greater than 1! In addition to this explicit discount factor, we'll need a function $\alpha$ to distribute this probability mass to the lower order n-grams.

This kind of backoff with discounting is also called **Katz backoff**. In Katz backoff we rely on a discounted probability *P\** if we've seen this n-gram before (i.e. if we have non-zero counts). Otherwise, we cursively back off to the Katz probability for the shorter-history (N-1)-gram. The probability for a backoff n-gram $P_{\mathrm{BO}}$ is thus computed as follows:

$$
P_{\mathrm{BO}}(w_n|w_{n-N+1}^{n-1}) = \begin{cases}P^*(w_n|w_{n-N+1}^{n-1}), & \mathrm{if}\,C(w_{n-N+1}^n > 0\\
\alpha(w_{n-N+1}^{n-1})P_{\mathrm{BO}}(w_n|w_{n-N+2}^{n-1}), & \mathrm{otherwise}.
\end{cases}
$$

Katz backoff is often combined with a smoothing method called **Good-Turing**. This involves quite detailed computation.